In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model



ModuleNotFoundError: No module named 'keras'

In [ ]:
# Load the temperature data from CSV
data = pd.read_csv('dataset.csv', header=None, names=['Temperature'])
data = data['Temperature'].values

# Split the data into timesteps
timesteps = 5
data = np.reshape(data, (-1, timesteps, 1))

# Split the data into train and test sets
train_data = data[:800]
test_data = data[800:]

# Define the model architecture
input_shape = (timesteps, 1)
latent_dim = 4

inputs = Input(shape=input_shape)
encoded = LSTM(latent_dim)(inputs)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(1, return_sequences=True)(decoded)

autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mae')

# Train the model
history = autoencoder.fit(train_data, train_data,
                          epochs=50,
                          batch_size=32,
                          validation_data=(test_data, test_data))

# Plot the training and validation loss
plt.hist(history.history['loss'], bins=10, label='Train MAE')
plt.hist(history.history['val_loss'], bins=10, label='Test MAE')
plt.legend()
plt.show()

# Make predictions on the test data
predictions = autoencoder.predict(test_data)

# Compute the anomaly score for each sample
anomaly_scores = np.mean(np.abs(predictions - test_data), axis=(1,2))

# Plot the anomaly score and original data for each sample
plt.plot(anomaly_scores, label='Anomaly score')
plt.plot(np.mean(np.abs(test_data), axis=(1,2)), label='Temperature')
plt.legend()
plt.show()